<div style="color: #fff7f7;
           display:fill;
           border-radius:10px;
           border-style: solid;
           border-color:#424949;
           text-align:center;
           background-color:#8ac6d1 ;
           font-size:20px;
           letter-spacing:0.5px;
           padding: 0.7em;
           text-align:left">  
<center> <h1>Classification coding challenge 🚩</h1> </center> 
</div>

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<div style="color: #fff7f7;
           display:fill;
           border-radius:10px;
           border-style: solid;
           border-color:#424949;
           text-align:center;
           background-color:#8ac6d1 ;
           font-size:20px;
           letter-spacing:0.5px;
           padding: 0.7em;
           text-align:left">  
<center> 1. Simple view of data  </center> 
</div>

In [ ]:
df_train = pd.read_csv('../input/cat-in-the-dat-ii/train.csv')
df_train.head()

In [ ]:
df_test = pd.read_csv('../input/cat-in-the-dat-ii/test.csv')
df_test.head()

- **bin----Duality**
- **nom----Nominal characteristics**
- **ord----Sequence feature**
- **day or month ----Date characteristics**

In [ ]:
df_train.duplicated().sum()

In [ ]:
round(df_train.isnull().sum()/df_train.shape[0],2)

- **null_num percent--3%**

<div style="color: #fff7f7;
           display:fill;
           border-radius:10px;
           border-style: solid;
           border-color:#424949;
           text-align:center;
           background-color:#8ac6d1 ;
           font-size:20px;
           letter-spacing:0.5px;
           padding: 0.7em;
           text-align:left">  
<center> 2. Dealing with missing values  </center> 
 <hr>
 <ul>
     <li>Use median, mean, and mode to fill numerical missing values.</li>
     <li>Use linear interpolation to fill in missing values.</li>
     <li>Use KNN algorithm to fill in numerical missing values.</li>
     <li>Randomly choose to fill in missing values of categorical features.</li>
     <li>For categorical features, add a new column to mark the missing values.</li>
     <li>Fill missing values of categorical features as Missing's special markers.</li>
     <li>Fill missing values into the category with the largest proportion.</li>
 </ul>
 <hr>
 <p style="color:"#fff7f7>💥My Choice：</p>
 <ul>
 <li>For Bin_0~4, fill missing values as Missing markers.</li>
 <li>For nom_0~9, fill in missing values with the smallest proportion of categories.</li>
 <li>For ord_0~5  fill missing values as Missing markers.</li>
 <li>For the numeric characteristics of the date column, use the median to fill.</li>  
 </ul>
</div>


In [ ]:
binc = ["bin_0","bin_1","bin_2","bin_3","bin_4"]
nomc = ["nom_0","nom_1","nom_2","nom_3","nom_4","nom_5","nom_6","nom_7","nom_8","nom_9"]
ordc = ["ord_0","ord_1","ord_2","ord_3","ord_4","ord_5"]
datec = ["day","month"]
misscol = binc + ordc

def fill_na(data,colist):
    
    if len(colist)==10:# fill nom_
        for col in colist:
            data[col] = data[col].fillna(data[col].value_counts(normalize=True).idxmin())
    elif len(colist)==11: #fill_binc+ordc
        for col in colist:
            data[col] = data[col].fillna("Missing")
    else:
        for col in colist:
            data[col] = data[col].fillna(data[col].median())
    return data

In [ ]:
df_train = fill_na(df_train,misscol)
df_train = fill_na(df_train,nomc)
df_train = fill_na(df_train,datec)
#---------------------------------
df_test = fill_na(df_test,misscol)
df_test = fill_na(df_test,nomc)
df_test = fill_na(df_test,datec)
print('df_train null_num: ',df_train.isnull().all().sum())
print('df_test null num: ',df_test.isnull().all().sum())
print(df_train.head())

<div style="color: #fff7f7;
           display:fill;
           border-radius:10px;
           border-style: solid;
           border-color:#424949;
           text-align:center;
           background-color:#8ac6d1 ;
           font-size:20px;
           letter-spacing:0.5px;
           padding: 0.7em;
           text-align:left">  
<center> 3. Encoding process  </center> 
<hr>
<p>💦Due to the use of algorithms such as XGboost, one-hot encoding is avoided as much as possible, and features are transformed according to label encoding.
   At the same time, for features containing letters and values, the corresponding letters and numbers will be extracted as new column identifiers.
    
</p>
<hr>
</div>

In [ ]:
for col in binc:
    print("This is {} unqiue values:".format(col))
    print(df_train[col].unique())
    print("-"*25)

- dealing wirh bin_columns

In [ ]:
bin_num = {"Missing":0.0,0.0:1.0,1.0:2.0}
bin_3 = {"Missing":0.0,"F":1.0,"T":2.0}
bin_4 = {"Missing":0.0,"N":1.0,"Y":2.0}
df_train["bin_0"] = df_train["bin_0"].map(bin_num)
df_train["bin_1"] = df_train["bin_1"].map(bin_num)
df_train["bin_2"] = df_train["bin_2"].map(bin_num)
df_train["bin_3"] = df_train["bin_3"].map(bin_3)
df_train["bin_4"] = df_train["bin_4"].map(bin_4)
#-------------------------------------------------
df_test["bin_0"] = df_test["bin_0"].map(bin_num)
df_test["bin_1"] = df_test["bin_1"].map(bin_num)
df_test["bin_2"] = df_test["bin_2"].map(bin_num)
df_test["bin_3"] = df_test["bin_3"].map(bin_3)
df_test["bin_4"] = df_test["bin_4"].map(bin_4)

In [ ]:
for col in ordc:
    print("This is {} unqiue values:".format(col))
    print(df_train[col].unique())
    print("-"*60)

In [ ]:
from sklearn.preprocessing import LabelEncoder
ord_0 = {"Missing":0.0}
ord_1 = {"Missing":0.0,"Novice":1.0,"Contributor":2.0,"Expert":3.0,"Master":4.0,"Grandmaster":5.0}
ord_2 = {"Missing":0.0,"Freezing":1.0,"Cold":2.0,"Warm":3.0,"Hot":4.0,"Boiling Hot":5.0,"Lava Hot":6.0}
ord_3 = {"Missing":0.0,"a":1.0,"b":2.0,"c":3.0,"d":4.0,"e":5.0,"f":6.0,"g":7.0,"h":8.0,"i":9.0,"j":10.0,"k":11.0,"l":12.0,"m":13.0,"n":14.0,"o":15.0}
ord_4 = {"Missing":0.0,"A":1.0,"B":2.0,"C":3.0,"D":4.0,"E":5.0,"F":6.0,"G":7.0,"H":8.0,"I":9.0,"J":10.0,"K":11.0,"L":12.0,
         "M":13.0,"N":14.0,"O":15.0,"P":16.0,"Q":17.0,"R":18.0,"S":19.0,"T":20.0,"U":21.0,"V":22.0,"W":23.0,"X":24.0,"Y":25.0,"Z":26.0}

lab = LabelEncoder()
df_train["ord_0"] = df_train["ord_0"].replace(ord_0)
df_train["ord_1"] = df_train["ord_1"].map(ord_1)
df_train["ord_2"] = df_train["ord_2"].map(ord_2)
df_train["ord_3"] = df_train["ord_3"].map(ord_3)
df_train["ord_4"] = df_train["ord_4"].map(ord_4)
lab.fit(df_train["ord_5"].values)
df_train['ord_5'] = lab.transform(df_train["ord_5"])


df_test["ord_0"] = df_test["ord_0"].replace(ord_0)
df_test["ord_1"] = df_test["ord_1"].map(ord_1)
df_test["ord_2"] = df_test["ord_2"].map(ord_2)
df_test["ord_3"] = df_test["ord_3"].map(ord_3)
df_test["ord_4"] = df_test["ord_4"].map(ord_4)
df_test['ord_5'] = lab.transform(df_test["ord_5"])


In [ ]:
for col in nomc:
    print("This is {} unqiue values:".format(col))
    print(df_train[col].unique())
    print("-"*60)

In [ ]:
lab1 = LabelEncoder()
lab1.fit(df_train["nom_0"])
df_train["nom_0"] = lab1.transform(df_train["nom_0"])
df_test["nom_0"] = lab1.transform(df_test["nom_0"])

lab2 = LabelEncoder()
lab2.fit(df_train["nom_1"])
df_train["nom_1"] = lab2.transform(df_train["nom_1"])
df_test["nom_1"] = lab2.transform(df_test["nom_1"])

lab3 = LabelEncoder()
lab3.fit(df_train["nom_2"])
df_train["nom_2"] = lab3.transform(df_train["nom_2"])
df_test["nom_2"] = lab3.transform(df_test["nom_2"])

lab4 = LabelEncoder()
lab4.fit(df_train["nom_3"])
df_train["nom_3"] = lab4.transform(df_train["nom_3"])
df_test["nom_3"] = lab4.transform(df_test["nom_3"])

lab5 = LabelEncoder()
lab5.fit(df_train["nom_4"])
df_train["nom_4"] = lab5.transform(df_train["nom_4"])
df_test["nom_4"] = lab5.transform(df_test["nom_4"])


- Regarding nom_5 to nom_9 as a password, calculate the length of distinct values.

In [ ]:
import re
df_train["nom_5"] = df_train["nom_5"].apply(lambda x:len(set(re.sub(r'\D',"",x))) + len(set(re.sub(r'\d',"",x))))
df_train["nom_6"] = df_train["nom_6"].apply(lambda x:len(set(re.sub(r'\D',"",x))) + len(set(re.sub(r'\d',"",x))))
df_train["nom_7"] = df_train["nom_7"].apply(lambda x:len(set(re.sub(r'\D',"",x))) + len(set(re.sub(r'\d',"",x))))
df_train["nom_8"] = df_train["nom_8"].apply(lambda x:len(set(re.sub(r'\D',"",x))) + len(set(re.sub(r'\d',"",x))))
df_train["nom_9"] = df_train["nom_9"].apply(lambda x:len(set(re.sub(r'\D',"",x))) + len(set(re.sub(r'\d',"",x))))
#---------------------------------------------------------------------------------------------------------------
df_test["nom_5"] = df_test["nom_5"].apply(lambda x:len(set(re.sub(r'\D',"",x))) + len(set(re.sub(r'\d',"",x))))
df_test["nom_6"] = df_test["nom_6"].apply(lambda x:len(set(re.sub(r'\D',"",x))) + len(set(re.sub(r'\d',"",x))))
df_test["nom_7"] = df_test["nom_7"].apply(lambda x:len(set(re.sub(r'\D',"",x))) + len(set(re.sub(r'\d',"",x))))
df_test["nom_8"] = df_test["nom_8"].apply(lambda x:len(set(re.sub(r'\D',"",x))) + len(set(re.sub(r'\d',"",x))))
df_test["nom_9"] = df_test["nom_9"].apply(lambda x:len(set(re.sub(r'\D',"",x))) + len(set(re.sub(r'\d',"",x))))
df_train.head()

<div style="color: #fff7f7;
           display:fill;
           border-radius:10px;
           border-style: solid;
           border-color:#424949;
           text-align:center;
           background-color:#8ac6d1 ;
           font-size:20px;
           letter-spacing:0.5px;
           padding: 0.7em;
           text-align:left">  
<center> 4. Feature Screening  </center> 
</div>

In [ ]:
plt.figure(figsize=(30,15))
corr = df_train.corr()
sns.heatmap(corr,annot=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
target = df_train[["target"]]
feature = df_train.drop(columns=["target","id"])
rf = RandomForestClassifier()
rf.fit(feature,target.values.reshape(-1,1))
im = pd.DataFrame()
plt.figure(figsize=(15,10))
g = sns.barplot(y=rf.feature_importances_,x=feature.columns,edgecolor="black",linewidth=3,color="#87ceed")
g.spines[['top', 'right','bottom']].set_visible(False)
g.axhline(0.02,color='r',linewidth=1)
plt.show()

- **According to the importance features of random forest screening, we plan to eliminate the feature columns from bin_0 to bin_2.**

In [ ]:
feature = feature.drop(columns=["bin_0","bin_1","bin_2"])
df_test = df_test.drop(columns=["bin_0","bin_1","bin_2"])

In [ ]:
target.value_counts(normalize=True)

- **It can be seen that the data is seriously unbalanced, so when dividing the data and the training data, use hierarchical cross-validation.**

<div style="color: #fff7f7;
           display:fill;
           border-radius:10px;
           border-style: solid;
           border-color:#424949;
           text-align:center;
           background-color:#8ac6d1 ;
           font-size:20px;
           letter-spacing:0.5px;
           padding: 0.7em;
           text-align:left">  
<center> 5 Model XGBoost and RandomForest  </center> 
</div>

In [ ]:
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.metrics import classification_report
import xgboost as xgb 
stf = StratifiedKFold(random_state=123,shuffle=False)
for train_index,test_index in stf.split(feature,target):
    x_train,x_test = feature.iloc[train_index],feature.iloc[test_index]
    y_train,y_test = target.iloc[train_index],target.iloc[test_index]
xgb = xgb.XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=5, min_child_weight=1, gamma=0,
                         subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, seed=27)
xgb.fit(x_train,y_train)
pre_xgb = xgb.predict(x_test)
xgb_prob = xgb.predict_proba(x_test)[:,1]

In [ ]:
print(classification_report(y_test,pre_xgb))

In [ ]:
rf_1 = RandomForestClassifier()
rf_1.fit(x_train,y_train)
rf_1_pre = rf_1.predict(x_test)
rf_prob = rf_1.predict_proba(x_test)[:,1]
print(classification_report(y_test,rf_1_pre))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
fig,ax = plt.subplots(1,2,figsize=(10,6))
cmxg = confusion_matrix(y_test,pre_xgb)  #xgboost
disp = ConfusionMatrixDisplay(cmxg)
disp.plot(ax=ax[0])


cm = confusion_matrix(y_test,rf_1_pre)  #RF
dis = ConfusionMatrixDisplay(cm)
dis.plot(ax=ax[1])
plt.show()

- **Xgboost on the left, random forest on the right.**
- **The actual number is 0, and the number of predictions is 0. Random forest is stronger than xgboost.**
- **The actual number is 1, and the number of predictions is 1. Xgboost forest is stronger than Random forest.**



In [ ]:
from sklearn.metrics import roc_auc_score
print("xgboost roc auc score:",roc_auc_score(y_test,xgb_prob))
print("RF roc auc score:",roc_auc_score(y_test,rf_prob))


<div style="color: #fff7f7;
           display:fill;
           border-radius:10px;
           border-style: solid;
           border-color:#424949;
           text-align:center;
           background-color:#8ac6d1 ;
           font-size:20px;
           letter-spacing:0.5px;
           padding: 0.7em;
           text-align:left">  
<center> 6 Stack  </center> 
</div>

In [ ]:
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression
from mlxtend.plotting import plot_decision_regions
import matplotlib.gridspec as gridspec

LR = LogisticRegression()
stack = StackingClassifier(classifiers=[xgb,rf_1],meta_classifier=LR)
stack.fit(x_train,y_train)


In [ ]:
stack_pre = stack.predict(x_test)
stack_prob = stack.predict_proba(x_test)[:,1]

In [ ]:
print(classification_report(y_test,stack_pre))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
fig,ax = plt.subplots(1,2,figsize=(16,6))


cmxg = confusion_matrix(y_test,pre_xgb)  #xgboost
disp = ConfusionMatrixDisplay(cmxg)
disp.plot(ax=ax[0])


cm = confusion_matrix(y_test,rf_1_pre)  #RF
dis = ConfusionMatrixDisplay(cm)
dis.plot(ax=ax[1])
plt.show()


scm = confusion_matrix(y_test,stack_pre)  #stack
distack = ConfusionMatrixDisplay(scm)
distack.plot()


In [ ]:
print("stack roc auc score:",roc_auc_score(y_test,stack_prob))

- **Score decreased**

In [ ]:
ids = df_test[["id"]]
test_data = df_test.drop(columns=["id"])
submisson_pre = stack.predict(test_data.values)
sub = pd.DataFrame()
sub["id"] = ids
sub["target"] = submisson_pre
sub.head()

In [ ]:
sub.to_csv("submission.csv",index=None)